We'll start with some magic that will make torch play nice with R on Talapas

In [ ]:
Sys.setenv(TORCH_HOME='/gpfs/projects/datascience/shared/R/torch/lantern/build/libtorch')
libdir='/gpfs/projects/datascience/shared/R/Data4ML'
.libPaths(libdir)
Sys.setenv(R_LIBS = paste(libdir, Sys.getenv("R_LIBS"), sep=.Platform$path.sep))

# Text Analysis for sequencing

We'll be playing around with the DeePromoter [https://github.com/egochao/DeePromoter](https://github.com/egochao/DeePromoter) dataset. This dataset contains 300bp sequences that bracket known transcription start sites (with the TSS located at position 250 out of 300) from the Eukaryotic Promoter Database (EPDnew). Their goal is to accurately predict sequences that act as promoter sites in eukaryotes for subsequent studies. We'll try a few methods to get a feel for utilizing raw text analysis

1. Feature engineering and Random Forests

2. A Deep Learning Approaching

In both cases, we'll need to create sequences to use non-promoter examples. We'll create random sequences, but there are other ways in the literature.




# Libraries
We'll use Torch for the dep learning and K-mer for  'feature engineering.'  

In [ ]:
library("torch")
library('kmer')
library('randomForest')


# Read in the Sequences File

In [ ]:
seq_data<-read.csv("/gpfs/projects/datascience/shared/deePromoter_data/hs_pos_TATA.txt",header=FALSE)

# Data Prep for Structured Features

We want to go from our string of letters to structured features. We'll start with getting kmer couts. The package 'kmer' were working with expect our sequencs as a matrix of characters. This means will need to break our strings apart

In [ ]:
head(seq_data)
pos_char<-strsplit(seq_data[,1],"")
head(pos_char)

Next were going to use the R command 
```R
sample
```
we'll use this to generate two kinds of non-promoter sequences
1. A random shuffle of our promoter sequences
2. Truly random sequences of As, Ts, Gs, and Cs


In [ ]:

n_neg=nrow(seq_data)
neg_char=c()

for( i in 1:n_neg){
    pos_sample=sample(length(pos_char),1)
    neg_sample=sample(pos_char[[pos_sample]],length(pos_char[[pos_sample]]))    
    neg_char=rbind(neg_char,neg_sample)
                   }  

rand_char=c()
for( i in 1:n_neg){
    r_sample=sample(c('A','T','G','C'),300,replace=T)
    rand_char<-rbind(rand_char,r_sample)
    }




Now we'll create our kmer counts, combine our data and add labels

In [ ]:
#Create Kmers
pos=data.frame(kcount(pos_char,k=4))
neg=data.frame(kcount(neg_char,k=4))
rand=data.frame(kcount(rand_char,k=4))


#Combine and add a Label
dataset=rbind(pos,neg,rand)
dataset$y <- c(rep(1,nrow(pos)),rep(0,nrow(neg)+nrow(rand)))

head(dataset)

# Remember from before we'll want to split the dataset into Train and Testing sets

In [ ]:
# Randomly Select training indices
train_index <- sample(seq_len(nrow(dataset)),size= nrow(dataset)*.8, replace=FALSE )

#Split the data
train <- dataset[train_index,]
test <- dataset[-train_index,]


x<-subset(train, select=-y)
y<-as.factor(train$y)

xtest<-subset(test, select=-y)
ytest<-as.factor(test$y)


# Fit a Random Forest

just like last time we can use a random forest and importance to learning what kmers might be useful for predicting promoters.

In [ ]:
rf=randomForest(x,y,xtest,ytest,ntree=100,importance=TRUE,keep.forest=TRUE)


In [ ]:
plot(rf)
print(rf)
importance(rf)

## The importance data is big so let's sort it

In [ ]:
imp <- as.data.frame(importance(rf))
imp[order(imp$MeanDecreaseAccuracy,decreasing = T),]

In [ ]:

best=which.max(imp$MeanDecreaseAccuracy)
#best=which(rownames(imp)=='TTTT')
name=rownames(imp)[best]


options(repr.plot.width=10,repr.plot.height=10)
ax<-0:30
prom<-hist(x[y==1,best],plot=FALSE,breaks=ax)
rand<-hist(x[y==0,best],plot=FALSE,breaks=ax)

c1=rgb(1,.1,.2,alpha=.30)
c2=rgb(1,1.,.2,alpha=.30)

plot(prom,col=c2,xlab=name,add=F,freq=FALSE)
plot(rand, col=c1,add=T,freq=FALSE)

legend(5,0.5,legend=c('Promoters','Random'),fill=c(c2,c1))




# Exercise try to predict new data
Let's see how well our model works on a new dataset
1. Process the datasets below into kmers
2. Make a histogram comparing the output of this dataset to random sequences


In [1]:
mouse_data<-read.csv("/gpfs/projects/datascience/shared/deePromoter_data/mm_pos_TATA.txt",header=FALSE)


In [ ]:
"Your Code Here"



# Try with a Deep Neural Network

The other way we can look at sequence data is with deep learning. This can be a lot more involve, but sometimes the payoff is worth it.

1. Instead of k-mers we're going to use the sequences mainly as is, however, we need to encode each character with a number. We'll use A=1,T=2,G=3,C=4. 

2. We'll need to write a deep learning model in torch. This can be a creative process with chances to improve your results, we'll use a Convolutional Neural Network or CNN.

3. We'll have to write a training loop, unlike our random forest we'll need to do this ourseleves.


# Convert our Data to Integers

In [ ]:
to_int <-function(x){
    out=c()
    for (i in 1:length(x)){
    if (x[i]=='A'){co=1}
    if (x[i]=='T'){co=2}
    if (x[i]=='G'){co=3}
    if (x[i]=='C'){co=4}
    out=c(out,co)
    }
return(out)
}


pos_mat <- do.call("rbind",pos_char)

all_char=rbind(pos_mat,neg_char,rand_char)
head(all_char)

all_char=apply(all_char,2,to_int)
head(all_char)




## Split the data like before

In [ ]:
dnn_train <- all_char[train_index,]
dnn_test <- all_char[-train_index,]
#print(nrow(dnn_test))

In [ ]:
head(all_char)

# Our DNN Model
There is a flow to most models in torch. 
1. Create your layers

   * **embedding** All text models will start with an 'Embedding' layer which maps each of our input integers into a a continous vector. It's just a learned matrix that returns the row of the each integer you put in
   * Intermediate layers - Convolutions, LSTM, etc
   * An activation function - This is applied at every layer it makes the model non-linear
   * Classification head - Fully connected layer with an outputsize of one

2. Define how they are put together to go from inputs to outputs ( this is called the foward pass)

In [ ]:
# Our Deep Neural Network has two functions initializtion and forward

net <- nn_module(
  "CNN model",

  initialize = function(vocab_size,emb_dim,
                        fc1_dim,
                        fc2_dim) {
    self$embedder <- nn_embedding(4,emb_dim)

    self$conv1<- nn_conv1d(emb_dim,50,10,stride=10)
    self$conv2<- nn_conv1d(50,50,10,stride=10)

    self$fc1 <- nn_linear(150, fc1_dim)
    self$fc2 <- nn_linear(fc1_dim, fc2_dim)  
    self$drop <- nn_dropout(0.2)
    self$output <- nn_linear(fc2_dim, 1)
    self$act <- nn_relu()
  },

  forward = function(x) {
    x <- self$embedder(x)

    x <- self$act(self$conv1(x$permute(c(1,3,2))))
    x<-self$drop(x)
    x <- self$act(self$conv2(x))
    x<-self$drop(x)
    x<-torch_flatten(x,start_dim=2)
    x<-self$act(self$fc1(x))
    x<-self$drop(x)
    x<-self$act(self$fc2(x))
    x<-self$drop(x)
    x<-self$output(x)
    x=torch_sigmoid(x)
    x=torch_squeeze(x)
    return(x)
  }
)

model=net(4,4,10,10)

# Batches 
DNN operate one batch at a time. A batch is just a fixed number of examples how many is refered to as the **batch_size**. **batch_sizes** vary from a few to hundreds or thosands 32 is a common default.

**Why Batches**? Computational requirments on DNN are high, normally too high to run an entire dataset at once.



In [ ]:


evaluate <- function(X,Y,batch_size){
    x_torch=torch_tensor(X,dtype=torch_int())
    y_torch=torch_tensor(Y,dtype=torch_float())-1 #here labels must be 0 or one but factors convert to 1/2
    dnn_pred=c()
    
    num_batch=floor(nrow(X)/batch_size)
    loss_v=c()
    print('Eval')
    model$eval()
    for (i in 0:num_batch ){
        end = (i+1)*batch_size
        if (end >nrow(X)){
            end=nrow(X)
            }
        batch_i=torch_tensor((i*batch_size+1):end)
        batch=x_torch[batch_i,]
        output=torch_squeeze(model(batch))
        loss <- criterion(output,y_torch[batch_i]) 
        loss_v=c(loss_v,c(loss$item()))
        dnn_pred<-c(dnn_pred,as.array(output))
        flush.console()    
    }
    model$train()
    
    return( list(pred=dnn_pred,loss=loss_v))
    }



In [ ]:
optimizer <- optim_adam(model$parameters,lr=1e-3)
criterion <- nn_bce_loss()

In [ ]:
batch_size <- 10
num_epochs <- 4


num_data_points <- length(y)
num_batches <- floor(num_data_points/batch_size)


for(current_epoch in 1:num_epochs){
  permute <- sample(num_data_points)
  data_x <- torch_tensor(dnn_train[permute,],dtype=torch_int())
  data_y <- torch_tensor(y[permute],,dtype=torch_float())-1
  
  running_loss=c()
  for(batch_idx in 1:num_batches){
    batch_i <- (batch_size*(batch_idx-1) + 1):(batch_idx*batch_size)
    batch=data_x[batch_i,]
    optimizer$zero_grad()
    output <- model(batch)
    loss <- criterion(output, data_y[batch_i]) #here labels must be 0 or one but factors convert to 1/2
    loss$backward()
    optimizer$step()

    running_loss=c(running_loss,c(loss$item()))
      
    if (length(running_loss)==50) {
        cat("train_loss",mean(running_loss), "epoch",current_epoch,"batch",batch_idx,"\n")
        running_loss=c()
        flush.console()
        }
    }
  ev_vals=evaluate(dnn_test,ytest,10)
  cat("Validation Loss",mean(ev_vals$loss),"Epoch",current_epoch,"\n")
      
  
}

In [ ]:
model$eval

In [ ]:
torch_save(model$state_dict(), 'model.pth')

In [ ]:
#model$load_state_dict(torch_load('/projects/datascience/shared/DATA4ML_weights/model.pth'))

In [ ]:
ev_vals=evaluate(dnn_test,ytest,10)
dnn_pred=ev_vals[[1]]


In [ ]:
sum(((dnn_pred>0.5) == (ytest==1)))/length(ytest)

In [ ]:
sum(dnn_pred[ytest==1]>0.5)/sum(ytest==1)

In [ ]:

options(repr.plot.width=10,repr.plot.height=10)
ax<-pretty(0:1,n=50)
prom<-hist(dnn_pred[ytest==1],plot=FALSE,breaks=ax)
rand<-hist(dnn_pred[ytest==0],plot=FALSE,breaks=ax)

c1=rgb(1,.1,.2,alpha=.30)
c2=rgb(1,1.,.2,alpha=.30)

plot(prom,col=c2,xlab='P(Prom|X)',freq=F)
plot(rand, col=c1,add=TRUE,freq=F)
legend(200,600,legend=c('Promoters','Random'),fill=c(c2,c1))



# Exercise 
Try our DNN on Mouse Data

In [ ]:
"Your Code Here"

